## Naive Bayes Classifier - Text Classification - Multinomial & Bernoulli

In this notebook you will apply the Naive Bayes algorithms for text classification.

Your tasks are marked with task numbers (e.g., Task 1). To get full credit you need to complete **ALL** tasks.



### Dataset: The 20 Newsgroups data set


The 20 newsgroups dataset comprises around 20,000 newsgroups posts on 20 topics split in two subsets: one for training (or development) and the other one for testing (or for performance evaluation). The 20 newsgroups collection has become a popular data set for experiments in text applications of machine learning techniques, such as text classification and text clustering. The split between the train and test set is based upon a messages posted before and after a specific date.

Following is a list of the 20 newsgroups, partitioned (more or less) according to subject matter:

- alt.atheism
- comp.graphics
- comp.os.ms-windows.misc
- comp.sys.ibm.pc.hardware
- comp.sys.mac.hardware
- comp.windows.x
- misc.forsale
- rec.autos
- rec.motorcycles
- rec.sport.baseball
- rec.sport.hockey
- sci.crypt
- sci.electronics
- sci.med
- sci.space
- soc.religion.christian
- talk.politics.guns
- talk.politics.mideast
- talk.politics.misc
- talk.religion.misc


You will normalize the documents, perform preprocessing and vectorize the features. Since the features are categorical, you will implement two different naive Bayes classifiers using Scikit-Learn. 
- Categorical features (binary valued) are modeled using the Multivariate Bernoulli distrubition 
- Categorical features (multi-valued) are modeled using the Multinomial distrubition 


## Steps for Classification:

1. Exploratory Data Analysis
2. Feature Extraction
   - a. Text Normalization (Stemming & Lemmatization)
   - b. Text Preprocessing (Tokenization, removing stop words, etc.)
   - c. Vectorization of the features
3. Model Selection by Hyperparameter Tuning
4. Train the Optimal Model
5. Analyzing Model Performance
6. Evaluate the Model on Test Data

In [6]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns

import nltk
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer 
from wordcloud import WordCloud

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_curve, precision_recall_curve, classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB

from sklearn.datasets import fetch_20newsgroups

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mattb\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Load Data

You will work on a partial dataset with only 4 categories out of the 20 available in the dataset:
- alt.atheism
- soc.religion.christian
- comp.graphics
- sci.med


The samples are shuffled randomly. This is useful if you wish to select only a subset of samples to quickly train a model and get a first idea of the results before re-training on the complete dataset later.

## Create Train and Test Sets 

In [7]:
categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med']

train_data = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
X_train = train_data.data
y_train = train_data.target


test_data = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)
X_test = test_data.data
y_test = test_data.target

# <font color=blue> 1. Exploratory Data Analysis</font>

## Quick Check of the Data

In [8]:
print("Target Names: ", train_data.target_names)

print("\nNumber of Training Examples: ", len(X_train))
print("Number of Training Labels: ", len(y_train))

print("Number of Test Examples: ",len(X_test))
print("Number of Test Labels: ", len(y_test))


print("\nPrint a Random Document:\n")
print(X_train[0])

Target Names:  ['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

Number of Training Examples:  2257
Number of Training Labels:  2257
Number of Test Examples:  1502
Number of Test Labels:  1502

Print a Random Document:

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton
Organization: The City University
Lines: 14

Does anyone know of a good way (standard PC application/PD utility) to
convert tif/img/tga files into LaserJet III format.  We would also like to
do the same, converting to HPGL (HP plotter) files.

Please email any response.

Is this the correct group?

Thanks in advance.  Michael.
-- 
Michael Collier (Programmer)                 The Computer Unit,
Email: M.P.Collier@uk.ac.city                The City University,
Tel: 071 477-8000 x3769                      London,
Fax: 071 477-8565                            EC1V 0HB.



## Class Distribution

#### Task 1: Compute class distribution in the following block (5 pts)

In [13]:
# print(y_train)
df = pd.DataFrame(y_train)
label_counts = df.iloc[:, -1].value_counts()
print(label_counts)

3    599
2    594
1    584
0    480
Name: 0, dtype: int64


## Visualization of the Class Distribution


#### Task 2: Generate visualization of the class distribution in the following block (5 pts)

# <font color=blue> 2. Feature Extraction </font>

## a) Text Normalization by Lemmatization

Stemming and Lemmatization are Text Normalization (or sometimes called Word Normalization) techniques in the field of Natural Language Processing that are used to prepare text, words, and documents for further processing.


#### Task 3: Lemmatize the training data. You may stem it as well, if it improves the classification accuracy. (10 pts)

# <font color=blue> 2. Feature Extraction </font>

## b) Text Preprocessing & c) Feature Vectorization

We can combine text preprocessing, feature vectorization and model training using the sklearn Pipeline object. This Pipeline object can be used for model selection and for training the optimal model. 

## <font color=blue> 3. Model Selection </font>


There are no hyperparameters in a NB model except the Laplace smoothing parameter alpha.

However, there are multiple hyperparameters for the CountVectorizer() and TfidfTransformer(). We need to select the best model based on the optimal values of these hyperparameters. This process is called hyper-parameter tuning.

For hyperparameter tuning, we will build a compund classifier using the sklearn Pipeline class. It will combine the CountVectorizer(), TfidfTransformer() and MultinomialNB() objects and will create a single object.

## Building a Pipeline for Hyperparameter Tuning


#### Task 4: Build a Pipeline object by combining CountVectorizer() and MultinomialNB() (5 pts)

## Hyperparameter Tuning

#### Task 5: Perform hyperparamer tuning for the following hyperparameters: (5 pts)
- CountVectorizer()
         -- ngram_range
         -- stop_words
- MultinomialNB()
        -- alpha
        
## **<font color=red size=5>Important:</font>**

The GridSearchCV takes an argument to define the scoring metric (performance measure). 

See the list of possible scoring functions:
https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

For multiclass classification, we may use "f1_micro" scoring function. The f1_micro function is the average of the F1 score of each class with weighting depending on the average parameter.

The macro-average ("f1_macro") will compute the metric independently for each class and then take the average (hence treating all classes equally), whereas a micro-average ("f1_micro") will aggregate the contributions of all classes to compute the average metric. In a multi-class classification setup, micro-average is preferable if you suspect there might be class imbalance (i.e you may have many more examples of one class than of other classes).

In the binary classification, "f1" score function can be used. We may also use the precision_score, recall_score, roc_auc_score functions.

In [ ]:
%%time



print("\nBest Score:")

print("\nOptimal Hyperparameter Values: ")



# <font color=blue> 4. Train the Optimal Multinomial Model </font>

#### Task 6: Using the optimal hyperparameter values, create the optimal model. Then, fit the model. (10 pts)
- Build a Pipeline object by combining CountVectorizer() and MultinomialNB()

# <font color=blue> 6. Evaluate the Model on Test Data </font>

#### Task 7:  Evaluate the model on test data and generate (10 pts)
- Confusion Matrix
- Precision
- Recall
- F1 score
- Classification Report


### Note: For multi-class classification, set the "average" attribute to "micro" for the following functions:
- precision_score
- recall_score
- f1_score

In [ ]:



print("\nTest Confusion Matrix:")


print("\nTest Precision:")


print("\nTest Recall:")


print("\nTest F1 Score:")


print("\nClassification Report:")


#  Multinomial NB: TF-IDF Model

#### Task 8: Implement the Multinomial model using the TF-IDF feature vectors (10 pts)
- Build a Pipeline object by combining CountVectorizer(), TfidfTransformer() and MultinomialNB()

## Evaluate the Model on Test Data 

#### Task 9: Evaluate the model on test data and generate (10 pts)
- Confusion Matrix
- Precision
- Recall
- F1 score
- Classification Report

In [ ]:




print("\nTest Confusion Matrix:")


print("\nTest Precision:")


print("\nTest Recall:")


print("\nTest F1 Score:")


print("\nClassification Report:")

## <font color=maroon> Observation on Multinomial Model With TF-IDF Feature Vectors </font>

We observe that both precision and recall decrease with TF-IDF feature vectors. 

#  Multivariate Bernoulli NB

#### Task 10: Implement the Multivariate Bernoulli Model (10 pts)
- Build a Pipeline object by combining CountVectorizer() and BernoulliNB()

### <font color=red> Note: </font>
The "binary" attribute of the CountVectorizer() object should be set to "True".

## Evaluate the Model on Test Data 


#### Task 11: Evaluate the model on test data and generate (10 pts)
- Confusion Matrix
- Precision
- Recall
- F1 score
- Classification Report

In [ ]:

print("\nTest Confusion Matrix:")


print("\nTest Precision:")


print("\nTest Recall:")


print("\nTest F1 Score:")


print("\nClassification Report:")

## <font color=maroon> Observation on Multivariate Bernoulli Model </font>

#### Task 12: Write a short account of your observation on the following aspects of your experimentation with 3 NB classifiers (10 pts):
- Impact of data normalization technique (did you observe performance improvement with lemmatization and/or stemming)
- Which classifier gave the best precision? Best recall? Best F1 Score? Explain their performance variance.
